In [19]:
import gradio as gr
import ollama

MAX_HISTORY = 10

MODEL = "llama3:8b"
SYSTEM_PROMPT = (
    "You are a knowledgeable and helpful AI assistant. Provide accurate, concise, and "
    "context-aware responses tailored to the user's needs. Maintain a professional, neutral tone, "
    "ask clarifying questions if needed, and avoid speculation or harmful content. "
    "Prioritize factual accuracy, explain concepts when necessary, and adapt to the user's "
    "familiarity with the topic. Return responses in the markdown format."
)
conversation = [{"role": "system", "content": SYSTEM_PROMPT}]


def llm(input_text):
    # Initialize the conversation if it's the first message
    global conversation

    # Append user input to the conversation
    conversation.append({"role": "user", "content": input_text})

    # Get the response from the LLM
    stream_out = ollama.chat(model=MODEL, messages=conversation, stream=True)

    # Collect the assistant's response
    output = ""
    for stream in stream_out:
        output += stream["message"]["content"]
        yield output

    # Append the assistant's response to the conversation
    conversation.append({"role": "assistant", "content": output})
    # Trim the message history if it exceeds the limit
    if len(conversation) > MAX_HISTORY:
        messages = conversation[-MAX_HISTORY:]


# Define the Gradio interface
interface = gr.Interface(
    fn=llm,
    inputs=[
        gr.Textbox(label="Your message:", lines=6),  # Input: User message
    ],
    outputs=[
        gr.Markdown(label="Response:"),  # Output: Assistant's response
    ],
    flagging_mode="never",
    theme=gr.Theme.from_hub("gstaff/sketch"),
    fill_width=True,
)


# Launch the Gradio interface
interface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7876
* Running on public URL: https://5bbabb882f02cc23c6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# TODO : Multi-user friendly